In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import date
import os 

driver = webdriver.Chrome()
driver.maximize_window()
location_name = 'London'
link = f'https://www.booking.com/searchresults.en-gb.html?ss={location_name}&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaGyIAQGYAQm4AQfIAQzYAQHoAQGIAgGoAgO4AueF3b0GwAIB0gIkMjFkYTVhYTMtZWM5ZC00ZmYyLTkzMDktZjUxN2IxMzVjZTdk2AIF4AIB&sid=e926d54c76bc20f9416c4f573131702a&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=209&dest_type=country&ac_position=1&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=fed6315912140504&ac_meta=GhBmZWQ2MzE1OTEyMTQwNTA0IAEoATICZW46BlRoYWlsYUAASgBQAA%3D%3D&checkin=2025-09-08&checkout=2025-09-09&group_adults=2&no_rooms=1&group_children=0&flex_window=1' 
driver.get(link)

In [5]:
batch_count = 1 
time.sleep(2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
while batch_count <= 10: 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    btn = driver.find_element(By.XPATH, '//*[@id="bodyconstraint-inner"]//button[@class="de576f5064 b46cd7aad7 d0a01e3d83 dda427e6b5 bbf83acb81 a0ddd706cc"]')
    btn.click()
    batch_count = batch_count + 1 

In [6]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
import lxml

In [8]:
header = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'}

In [9]:
soup = BeautifulSoup(driver.page_source , 'lxml')

In [10]:
def get_locator_text_or_element(locator, get_locator=False): 
    if locator: 
        if get_locator:
            return locator
        return locator.text 
    else: 
        return None 

In [ ]:
rows = []
location_name = 'United Kingdom'
for i in soup.find(role="list").find_all(role="listitem"):
    title = get_locator_text_or_element(i.find(attrs={"data-testid":"title"}))
    review_text = get_locator_text_or_element(i.find(attrs={"class":"f63b14ab7a f546354b44 becbee2f63"}))
    sustainability_cert = get_locator_text_or_element(i.find(class_="fff1944c52 f546354b44 b79604af7c"))
    address = get_locator_text_or_element(i.find(attrs={"data-testid":"address"}))
    total_reviews = get_locator_text_or_element(i.find(attrs={"class":"fff1944c52 fb14de7f14 eaa8455879"}))
    review_score = get_locator_text_or_element(i.find(attrs={"class":"f63b14ab7a dff2e52086"}))
    price_undiscounted_amount = get_locator_text_or_element(i.find(attrs={"class":"fff1944c52 d68334ea31 ab607752a2"}))
    price_discounted_amount = get_locator_text_or_element(i.find(attrs={"data-testid":"price-and-discounted-price"}))
    taxes_and_charges = get_locator_text_or_element(i.find(attrs={"data-testid":"taxes-and-charges"}))
    booking_room_feature = get_locator_text_or_element(i.find(attrs={"class":"fff1944c52 f254df5361"}))
    highly_rated_luxurious_stay = get_locator_text_or_element(i.find(attrs={"class":"f88cca4051"}))
    
    stars_list = get_locator_text_or_element(i.find(attrs={"data-testid":"rating-stars"}), get_locator=True)
    if stars_list: 
        stars_list_count = len(stars_list.find_all(class_="e03979cfad"))
    else: 
        stars_list_count = None 
    
    booking_room_details = i.find(attrs={"class":"d1e8dce286"})    
    if (i.find(attrs={"data-testid":"property-card-deal"})):
        deal_type_description = (i.find(attrs={"data-testid":"property-card-deal"}))['aria-label']
        deal_type = (i.find(attrs={"data-testid":"property-card-deal"})).find('span').text 
    else: 
        deal_type = None 
        deal_type_description = None 
    red_text = get_locator_text_or_element(booking_room_details.find(class_='b7d3eb6716'))
    free_cancellation = get_locator_text_or_element(booking_room_details.find("div", class_='fff1944c52 cff4a33cd8'))
    breakfast_included = get_locator_text_or_element(booking_room_details.find("span", class_='d651523034'))
    room_entire_text = get_locator_text_or_element(booking_room_details.find("div", class_='ba43cae375'))
    image_link = get_locator_text_or_element(i.find(attrs={"data-testid":"property-card-container"}), get_locator=True).find('img')['src']

    rows.append([title,review_text,sustainability_cert,address,total_reviews,review_score,price_undiscounted_amount,price_discounted_amount,taxes_and_charges,booking_room_feature,red_text,room_entire_text,breakfast_included,free_cancellation,deal_type, deal_type_description,stars_list_count,highly_rated_luxurious_stay,image_link, location_name])

df= pd.DataFrame(rows, columns=['title','review_text','sustainability_cert','address','total_reviews','review_score','price_undiscounted_amount','price_discounted_amount','taxes_and_charges','booking_room_feature','red_text','room_entire_text','breakfast_included','free_cancellation','deal_type','deal_type_description','stars_list_count','highly_rated_luxurious_stay','image_link','location_name'])

In [3]:
dataset_name = 'hotels_dataset_'+str(date.today())+'.parquet'
df.to_parquet(dataset_name)

NameError: name 'df' is not defined

In [12]:
driver.close()

In [2]:
import boto3 as b
s3_client = b.client('s3')

In [4]:
s3_client.upload_file(Filename=os.getcwd()+'/'+dataset_name, Bucket='landing-data-bucket-5f593a', Key='hotels/'+dataset_name)